# 1. 사전 준비

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 패키지 설치하기
pip 명령어로 의존성 있는 패키지를 설치합니다.

In [2]:
%cd /content/drive/MyDrive/recipekogpt2/

/content/drive/MyDrive/recipekogpt2


In [ ]:
!pip install -r requirements.txt

     |████████████████████████████████| 808kB 7.8MB/s 
     |████████████████████████████████| 2.3MB 51.5MB/s 
     |████████████████████████████████| 61kB 9.1MB/s 
     |████████████████████████████████| 10.6MB 56.9MB/s 
     |████████████████████████████████| 829kB 62.3MB/s 
     |████████████████████████████████| 122kB 63.5MB/s 
     |████████████████████████████████| 645kB 71.7MB/s 
     |████████████████████████████████| 276kB 70.5MB/s 
     |████████████████████████████████| 3.3MB 41.9MB/s 
     |████████████████████████████████| 901kB 59.2MB/s 
     |████████████████████████████████| 102kB 12.5MB/s 
     |████████████████████████████████| 1.3MB 63.6MB/s 
     |████████████████████████████████| 143kB 70.9MB/s 
     |████████████████████████████████| 296kB 66.0MB/s 
  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491070 sha256=d2424af9b39e56d38ca0ed740226813c2ab6c24ff6456a08d5c26d925635df28
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dc

### 각종 설정
모델 하이퍼파라메터(hyperparameter)와 저장 위치 등 설정 정보를 선언합니다.

In [ ]:
import torch
from ratsnlp.nlpbook.generation import GenerationTrainArguments
args = GenerationTrainArguments(
    pretrained_model_name="skt/kogpt2-base-v2",
    downstream_corpus_name="recipegpt",
    downstream_corpus_root_dir='/content/drive/MyDrive/recipekogpt2/data',
    downstream_model_dir="/content/drive/MyDrive/recipekogpt2/model_checkpoints",
    max_seq_length=120,
    batch_size= 8 if torch.cuda.is_available() else 4,
    learning_rate=5e-5,
    epochs=1,
    tpu_cores=0 if torch.cuda.is_available() else 8,
    seed=7,
)

### 랜덤 시드 고정
학습 재현을 위해 랜덤 시드를 고정합니다.

In [ ]:
from ratsnlp import nlpbook
nlpbook.set_seed(args)

set seed: 7


### 로거 설정
메세지 출력 등을 위한 logger를 설정합니다.

In [ ]:
nlpbook.set_logger(args)

INFO:ratsnlp:Training/evaluation parameters GenerationTrainArguments(pretrained_model_name='skt/kogpt2-base-v2', downstream_task_name='sentence-generation', downstream_corpus_name='recipegpt', downstream_corpus_root_dir='/content/drive/MyDrive/encodded_data', downstream_model_dir='/content/drive/MyDrive/ratsgockpt', max_seq_length=200, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=1, batch_size=32, cpu_workers=4, fp16=False, tpu_cores=0)


### 토크나이저 준비
토큰화를 수행하는 토크나이저를 선언합니다. 이때, 데이터를 만들 때 사용했던 token의 리스트(unused0~unused5)도 토크나이저가 잘 인식할 수 있도록 추가해줍니다. 각 토큰은 순서대로 요리이름의 시작과 끝, 재료의 시작과 끝, 레시피 본문의 시작과 끝을 나타냅니다.

In [ ]:
tokens_list = ['<unused0>','<unused1>','<unused2>','<unused3>','<unused4>','<unused5>']
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>', additional_special_tokens = tokens_list) 
tokenizer.encode("<unused0><unused1><unused99>김상병$")

[9, 10, 108, 23201, 7648, 379]

In [ ]:
tokenizer.tokenize('<unused0><unused1><unused99>김상병$')

['<unused0>', '<unused1>', '<unused99>', '▁김상', '병', '$']

모델이 51200차원의 임베딩으로 훈련되었기 때문에 혹시 사이즈가 맞지 않는다면 special token을 잘못 추가한 것이며, 추후 훈련할 때 CUDA error가 나타나기 때문에 수정해야 합니다.

In [ ]:
tokenizer.vocab_size

51200

# 2. 학습데이터 구축
학습데이터를 만듭니다.
### training data 구축

In [ ]:
from ratsnlp.nlpbook.generation import NsmcCorpus, GenerationDataset
from torch.utils.data import DataLoader, SequentialSampler, RandomSampler
corpus = NsmcCorpus()
train_dataset = GenerationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="train",
)
train_dataloader = DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    sampler=RandomSampler(train_dataset, replacement=False),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

INFO:ratsnlp:Creating features from dataset file at /content/drive/MyDrive/encodded_data/recipegpt
INFO:ratsnlp:loading train data... LOOKING AT /content/drive/MyDrive/encodded_data/recipegpt/recipegpt_train.txt
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences [took 5.839 s]
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:sentence: <unused0>누룽지 두부 계란죽 <unused1> <unused2>[$'$애$호$박$'$,$ $'$표$고$버$섯$'$,$ $'$당$근$ $누$룽$지$ $누$룽$지$'$,$ $'$순$두$부$'$,$ $'$달$걀$'$,$ $'$참$기$름$'$,$ $'$소$금$'$,$ $'$참$깨$'$,$ $'$흰$ $후$추$'$]$ <unused3> <unused4>깨끗이 씻어 손질한 애호박 당근과 기둥을 뗀 표고버섯을 잘게 다지듯이 썬다. 누룽지는 1 정도로 잘게 부숴준다. 냄비에 참기름을 두르고 썰어 놓은 채소를 볶다가 누룽지와 물을 넣고 끓인다. 누룽지가 살짝 퍼지면 순두부를 넣고 흰 후추와 소금을 넣는다. 죽이 끓으면 달걀을 풀어 넣고 한 소끔 끓여낸 후 참깨를 뿌려 마무리한다. <unused5>
INFO:ratsnlp:tokens: <unused0> ▁누 룽 지 ▁두부 ▁계 란 죽 ▁ <unused1> ▁ <unused2> ▁[ $ ' $ 애 $ 호 $ 박 $ ' $ , $ ▁ $ ' $ 표 $ 고 $ 버 $ 섯 $ ' $ , $ ▁ $ ' $ 당 $ 근 $ ▁ $ 누 $ 룽 $ 지 $ ▁ $ 누 $ 룽 $ 지 $ ' $ , $ ▁ $ ' $ 순 $ 두 $ 부 $ ' $ , $ ▁ $ ' $ 달 $ 걀 $ '

### validation data 구축
학습 중에 평가할 테스트 데이터를 구축합니다.

In [ ]:
val_dataset = GenerationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="test",
)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=args.batch_size,
    sampler=SequentialSampler(val_dataset),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)


INFO:ratsnlp:Creating features from dataset file at /content/drive/MyDrive/encodded_data/recipegpt
INFO:ratsnlp:loading test data... LOOKING AT /content/drive/MyDrive/encodded_data/recipegpt/recipegpt_test.txt
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences [took 0.851 s]
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:sentence: <unused0>치커리샐러드와 올리브 마늘 소스 <unused1> <unused2>[$'$치$커$리$ $샐$러$드$치$커$리$ $줄$기$'$,$ $'$적$양$배$추$'$,$ $'$양$파$'$,$ $'$당$근$ $올$리$브$마$늘$ $드$레$싱$올$리$브$유$'$,$ $'$식$초$'$,$ $'$설$탕$'$,$ $'$마$늘$ $쪽$'$]$ <unused3> <unused4>마늘은 곱게 다진다. 올리브유 식초 설탕 다진 마늘을 섞어 거품기로 충분히 저어주 어 올리브 마늘 드레싱을 만든다. 치 커리는 싱싱하게 찬물에 담갔다가 물기를 뺀 후 한 입 크기로 자르고 적양배추 양파 당근은 곱게 채를 썬다. 접시에 준비한 치 커리 적 양배추 양파 당근을 담고 올리브 마늘 드레싱을 뿌린다. <unused5>
INFO:ratsnlp:tokens: <unused0> ▁치 커 리 샐 러 드와 ▁올리브 ▁마늘 ▁소 스 ▁ <unused1> ▁ <unused2> ▁[ $ ▁' $ ▁치 $ ▁커 $ ▁리 $ ▁ $ ▁샐 $ ▁러 $ ▁드 $ ▁치 $ ▁커 $ ▁리 $ ▁ $ ▁줄 $ ▁기 $ ▁' $ ▁, $ ▁ $ ▁' $ ▁적 $ ▁양 $ ▁배 $ ▁추 $ ▁' $ ▁, $ ▁ $ ▁' $ ▁양 $ ▁파 $ ▁' $ ▁, $ ▁ 

# 3. 학습 준비

### 모델 초기화
프리트레인이 완료된 GPT2 모델을 읽고, 문장 생성 모델을 초기화합니다.

In [ ]:
from transformers import GPT2LMHeadModel
model = GPT2LMHeadModel.from_pretrained(
    args.pretrained_model_name
)

Task와 Trainer를 준비합니다.

In [ ]:
train_dataloader = DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    sampler=RandomSampler(train_dataset, replacement=False),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

val_dataloader = DataLoader(
    val_dataset,
    batch_size=args.batch_size,
    sampler=SequentialSampler(val_dataset),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)


In [ ]:
from ratsnlp.nlpbook.generation import GenerationTask
task = GenerationTask(model, args)

traininer 선언 : checkpoint부터 이어서 학습한다면, file exists 경고가 뜨지만 args를 도중에 바꾸지 않는다면 문제는 없습니다.

In [ ]:
trainer = nlpbook.get_trainer(args)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


# 4. 학습
준비한 데이터와 모델로 학습을 시작합니다. 학습 결과물(체크포인트)은 미리 연동해둔 구글 드라이브의 준비된 위치(`/recipekogpt2/model_checkpoints`)에 저장됩니다.

In [ ]:
!nvidia-smi

Tue Jun 15 17:23:28 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P0    41W / 250W |  16259MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
trainer.fit(
    task,
    train_dataloader=train_dataloader,
    val_dataloaders=val_dataloader,
)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type            | Params
------------------------------------------
0 | model | GPT2LMHeadModel | 125 M 
------------------------------------------
125 M     Trainable params
0         Non-trainable params
125 M     Total params
500.656   Total estimated model params size (MB)
